In [ ]:
from langchain.chat_models import AzureChatOpenAI

# assuming LLM api keys have been set in the environment

llm_35 = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    temperature=0,
    request_timeout=20,
    max_retries=1,
    client=None,
)

llm_4 = AzureChatOpenAI(
    deployment_name="gpt-4",
    temperature=0,
    request_timeout=20,
    max_retries=1,
    client=None,
)

llm_35.predict('What American cartoonist is the creator of Andy Lippincott?')

In [ ]:
from langchain.prompts.prompt import PromptTemplate

_SIMPLE_PROMPT = """{context}"""


OTHER_PROMPT = PromptTemplate(
    input_variables=["context"],
    template=_SIMPLE_PROMPT,
)

In [ ]:
from personalizer import *
from graph_feedback_chain import *

graph_chain = GraphFeedbackChain.from_llms({"gpt-35-turbo": llm_35, "gpt-4": llm_4}, prompt=OTHER_PROMPT)


In [ ]:
inputs={"selected": "gpt-35-turbo", "context":"What American cartoonise is the creator of Andy Lippincott?"}

graph_chain.run(inputs)

In [ ]:

chain = PersonalizerChain.from_chain(
    llm_chain=graph_chain,
    vw_workspace_type=PersonalizerChain.Type.CONTEXTUAL_BANDITS,
    model_save_dir="./gf_models",  # where to save the model checkpoints
    # response_checker = LLMResponseCheckerForCB(llm=llm_35),
    vw_cmd=['--cb_explore_adf', '--quiet', '--interactions=::', '--coin', '--graph_feedback'],
    prompt=OTHER_PROMPT,    
)

In [ ]:
inputs={"context":"What American cartoonise is the creator of Andy Lippincott?", "actions":["gpt-35-turbo", "gpt-4"]}

chain.run(inputs)